This notebook is a simplified notebook version of the implementation bayesian alignment of LLM


# all the data and files are assumed to be inside /content/ ..

# TODO: probably generate a loss graph for different loss function.

In [ ]:

PARENT = "/kaggle/input/preference-learning/" #directory including the pre-processed data and config files
OUTPUT = "/kaggle/working/" #output directory where model, losses, and configs will be saved

In [ ]:
from transformers import (AutoTokenizer, AutoModel, AutoConfig)
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
#@title bert model commons
import math
import inspect
import torch
import torch.nn as nn
from typing import (List, Optional, Tuple, Union, Callable)
from transformers import (PreTrainedTokenizer, PreTrainedModel, PretrainedConfig)
from transformers.models.bert import (BertConfig, BertModel, BertLayer, )
from transformers.models.bert.modeling_bert import (BertAttention, BertSelfAttention, )
from transformers.modeling_outputs import (BaseModelOutputWithPoolingAndCrossAttentions,
                                           BaseModelOutputWithPastAndCrossAttentions)


class BaseModel(PreTrainedModel):
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)
        self.toker = None

    def tie_tokenizer(self, toker: PreTrainedTokenizer):
        self.toker = toker
        if len(self.toker) > self.toker.vocab_size:
            self.resize_token_embeddings(len(self.toker))


class PrefixBertSelfAttention(BertSelfAttention):
    def __init__(self, config):
        super().__init__(config)

    def forward(
            self,
            hidden_states: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
            prefix: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor]:
        """
        the main bert self-attention module acts as both self and cross-attention.

        query = query(hidden_states)
        if cross-attention and past_key_values:
          key, value = past_key_value[0], [1]
        elif cross-attention:
          key, value = key/value(encoder_hidden_states)
        elif past_key_values != None:
          key, value = past_key_value[0]/[1] + key/value(hidden_states)  ##concat
        else:
          key, value = key/value(hidden_states)



        """
        batch_size, seq_length = hidden_states.shape[:2]
        real_seq_length = seq_length

        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        use_cache = past_key_value is not None
        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        if prefix is not None:
            key_layer = torch.cat([prefix["prev_key"], key_layer], dim=2)
            value_layer = torch.cat([prefix["prev_value"], value_layer], dim=2)  # TODO: Chen

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            query_length, key_length = query_layer.shape[2], key_layer.shape[2]
            if use_cache:
                position_ids_l = torch.tensor(key_length - 1, dtype=torch.long, device=hidden_states.device).view(
                    -1, 1
                )
            else:
                position_ids_l = torch.arange(query_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(key_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r

            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        #########################################
        if prefix is not None:
            assert key_layer.shape[2] > attention_mask.shape[3]
            assert prefix["prev_key_padding_mask"].shape[1] == key_layer.shape[2] - real_seq_length
            assert attention_mask.shape[3] == real_seq_length
            attention_mask = torch.cat([prefix["prev_key_padding_mask"].float().unsqueeze(1).unsqueeze(2).
                                       expand(-1, -1, attention_mask.shape[2], -1) * -10000.0, attention_mask], dim=3)
        #########################################

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs


class PrefixBertAttention(BertAttention):
    """
    just applying the attention above and the last linear and residual in a normal attention layer.
    outputing attention probs as second argument
    """
    def __init__(self, config, position_embedding_type=None):
        super().__init__(config, position_embedding_type)
        self.self = PrefixBertSelfAttention(config)

    def forward(
            self,
            hidden_states: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
            prefix: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor]:
        self_outputs = self.self(
            hidden_states,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
            prefix,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs


class PrefixBertLayer(BertLayer):
    def __init__(self, config):
        super().__init__(config)
        self.attention = PrefixBertAttention(config)
        if self.add_cross_attention:
            if not self.is_decoder:
                raise ValueError(f"{self} should be used as a decoder model if cross attention is added")
            self.crossattention = PrefixBertAttention(config, position_embedding_type="absolute")

    def forward(
            self,
            hidden_states: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
            encoder_prefix: Optional[torch.Tensor] = None,
            decoder_prefix: Optional[torch.Tensor] = None,
            cross_attn_prefix: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            past_key_value=self_attn_past_key_value,
            prefix=decoder_prefix if self.is_decoder else encoder_prefix,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        if self.is_decoder:
            outputs = self_attention_outputs[1:-1]
            present_key_value = self_attention_outputs[-1]
        else:
            outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        cross_attn_present_key_value = None
        if self.is_decoder and encoder_hidden_states is not None:
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with cross-attention layers"
                    " by setting `config.add_cross_attention=True`"
                )

            # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            cross_attention_outputs = self.crossattention(
                attention_output,
                attention_mask=attention_mask,
                head_mask=head_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                past_key_value=cross_attn_past_key_value,
                output_attentions=output_attentions,
                prefix=cross_attn_prefix,
            )
            attention_output = cross_attention_outputs[0]
            outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

            # add cross-attn cache to positions 3,4 of present_key_value tuple
            cross_attn_present_key_value = cross_attention_outputs[-1]
            present_key_value = present_key_value + cross_attn_present_key_value

        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs

        # if decoder, return the attn key/values as the last output
        if self.is_decoder:
            outputs = outputs + (present_key_value,)

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output


class PrefixBertEncoder(nn.Module):
    def __init__(self, config: BertConfig):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([PrefixBertLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
            self,
            hidden_states: torch.Tensor,
            past_prompt: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = False,
            output_hidden_states: Optional[bool] = False,
            return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        prefix_l = len(self.layer) - len(past_prompt)
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            encoder_prefix = None
            decoder_prefix = None
            cross_attn_prefix = None
            if i >= prefix_l and past_prompt:
                encoder_prefix = past_prompt[i - prefix_l].get('encoder_prompt', None)
                decoder_prefix = past_prompt[i - prefix_l].get('decoder_prompt', None)
                cross_attn_prefix = past_prompt[i - prefix_l].get('cross_attention_prompt', None)

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                    encoder_prefix=encoder_prefix,
                    decoder_prefix=decoder_prefix,
                    cross_attn_prefix=cross_attn_prefix,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


class PrefixBertModel(BertModel):
    def __init__(self, config: BertConfig, add_pooling_layer=True):
        super().__init__(config, add_pooling_layer)
        self.encoder = PrefixBertEncoder(config)

    def forward(
            self,
            input_ids: Optional[torch.Tensor] = None,
            past_prompt: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            token_type_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            head_mask: Optional[torch.Tensor] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            encoder_hidden_states: Optional[torch.Tensor] = None,
            encoder_attention_mask: Optional[torch.Tensor] = None,
            past_key_values: Optional[List[torch.FloatTensor]] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            past_prompt=past_prompt,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


def apply_chunking_to_forward(
        forward_fn: Callable[..., torch.Tensor], chunk_size: int, chunk_dim: int, *input_tensors
) -> torch.Tensor:
    """
    This function chunks the `input_tensors` into smaller input tensor parts of size `chunk_size` over the dimension
    `chunk_dim`. It then applies a layer `forward_fn` to each chunk independently to save memory.
    If the `forward_fn` is independent across the `chunk_dim` this function will yield the same result as directly
    applying `forward_fn` to `input_tensors`.
    Args:
        forward_fn (`Callable[..., torch.Tensor]`):
            The forward function of the model.
        chunk_size (`int`):
            The chunk size of a chunked tensor: `num_chunks = len(input_tensors[0]) / chunk_size`.
        chunk_dim (`int`):
            The dimension over which the `input_tensors` should be chunked.
        input_tensors (`Tuple[torch.Tensor]`):
            The input tensors of `forward_fn` which will be chunked
    Returns:
        `torch.Tensor`: A tensor with the same shape as the `forward_fn` would have given if applied`.
    Examples:
    ```python
    # rename the usual forward() fn to forward_chunk()
    def forward_chunk(self, hidden_states):
        hidden_states = self.decoder(hidden_states)
        return hidden_states
    # implement a chunked forward function
    def forward(self, hidden_states):
        return apply_chunking_to_forward(self.forward_chunk, self.chunk_size_lm_head, self.seq_len_dim, hidden_states)
    ```"""

    assert len(input_tensors) > 0, f"{input_tensors} has to be a tuple/list of tensors"

    # inspect.signature exist since python 3.5 and is a python method -> no problem with backward compatibility
    num_args_in_forward_chunk_fn = len(inspect.signature(forward_fn).parameters)
    if num_args_in_forward_chunk_fn != len(input_tensors):
        raise ValueError(
            f"forward_chunk_fn expects {num_args_in_forward_chunk_fn} arguments, but only {len(input_tensors)} input "
            "tensors are given"
        )

    if chunk_size > 0:
        tensor_shape = input_tensors[0].shape[chunk_dim]
        for input_tensor in input_tensors:
            if input_tensor.shape[chunk_dim] != tensor_shape:
                raise ValueError(
                    f"All input tenors have to be of the same shape: {tensor_shape}, "
                    f"found shape {input_tensor.shape[chunk_dim]}"
                )

        if input_tensors[0].shape[chunk_dim] % chunk_size != 0:
            raise ValueError(
                f"The dimension to be chunked {input_tensors[0].shape[chunk_dim]} has to be a multiple of the chunk "
                f"size {chunk_size}"
            )

        num_chunks = input_tensors[0].shape[chunk_dim] // chunk_size

        # chunk input tensor into tuples
        input_tensors_chunks = tuple(input_tensor.chunk(num_chunks, dim=chunk_dim) for input_tensor in input_tensors)
        # apply forward fn to every tuple
        output_chunks = tuple(forward_fn(*input_tensors_chunk) for input_tensors_chunk in zip(*input_tensors_chunks))
        # concatenate output at same dimension
        return torch.cat(output_chunks, dim=chunk_dim)

    return forward_fn(*input_tensors)


In [ ]:
#@title model itself


from transformers import BertForSequenceClassification #BertConfig, BertModel,
def ce_loss(preds, prior):
    return -(preds * prior).sum(1), torch.Tensor([0])


def comb_loss(preds, prior):
    return -(preds.softmax(1) * prior / prior.sum(1).unsqueeze(1)).sum(1).log(), torch.Tensor([0])

def comp_r (preds, priors, eps=1e-9):
    logprior = priors.clamp(min=eps).log()
    r = (preds.log_softmax(0) + logprior).log_softmax(1)
    return r

def comp_kl_r(preds, mix, prior=False):
    if prior:
        preds = preds.clamp(min=eps).log()
    t1 = -(mix * preds.exp()).sum(1)
    t2 = -(preds * preds.exp()).sum(1)
    return t1.mean() - t2.mean()

def qr_loss(preds, prior, eps=1e-9):
    logprior = prior.clamp(min=eps).log()
    r = (preds.log_softmax(0) + logprior).log_softmax(1)
    return -(r * preds.exp()).sum(1), -(preds * preds.exp()).sum(1)


def rq_loss(preds, prior, eps=1e-9):
    logprior = prior.clamp(min=eps).log()
    r = (preds.log_softmax(0) + logprior).log_softmax(1)
    return -(preds * r.exp()).sum(1), -(r * r.exp()).sum(1)


class Model(BaseModel, BertForSequenceClassification):
    def __init__(self, config: BertConfig, args):
        super().__init__(config)
        self.n_embd = self.config.hidden_size
        self.mid_dim = args.d_prefix
        self.match_n_layer = self.config.num_hidden_layers // 2
        self.match_n_head = self.config.num_attention_heads
        self.match_n_embd = self.config.hidden_size // self.config.num_attention_heads
        self.hidden_size = self.config.hidden_size
        self.class_num = args.class_num
        self.loss_fn = args.loss_fn

        # The Multi prefix modules!
        # The task-prefix modules from all specific tasks
        self.prefix_names = ["posterior"]
        self.preseqlen = args.preseqlen
        self.prefix_tokens = torch.arange(self.preseqlen).long()
        self.input_tokens = torch.arange(self.preseqlen).long()
        self.multi_prefix = nn.ModuleDict(
            {
                name: nn.ModuleDict(
                    {
                        "wte_enc": nn.Embedding(self.preseqlen, self.n_embd),
                        "control_trans_enc": nn.Sequential(
                            nn.Linear(self.n_embd, self.mid_dim),
                            nn.Tanh(),
                            nn.Linear(self.mid_dim, self.match_n_layer * 2 * self.match_n_head * self.match_n_embd),
                        ),
                        "mlp": nn.Sequential(
                            nn.Linear(config.hidden_size, self.mid_dim),
                            nn.Tanh(),  # nn.LeakyReLU(),
                            nn.Linear(self.mid_dim, self.class_num),
                        ),
                    }
                )
                for name in self.prefix_names
            }
        )
        self.prefix_dropout = nn.Dropout(args.prefix_dropout)
        self.bert = PrefixBertModel(config)
        self.classifier = None

    def get_distribution(self, labels, bsz=None, smooth=0.1):
        """ Convert labels to label distribution.

        :param labels: (bsz, )
        :param bsz: batch size
        :param smooth: smoothing factor
        :return labels_dist: (bsz, class_num)

        """
        class_num = self.class_num
        labels_dist = torch.zeros(bsz, class_num + 1).to(labels.device)
        one_tensor = torch.ones_like(labels_dist)
        label_mask = labels == -100
        labels = labels.masked_fill(label_mask, class_num)
        labels_dist = labels_dist.scatter_add(1, labels, one_tensor)
        labels_dist = labels_dist[:, :-1] + smooth
        assert labels_dist.shape == (bsz, class_num)
        return labels_dist / labels_dist.sum(-1, keepdim=True)

    def get_prompt(self, name, bsz=None, device="cuda"):
        old_bsz = bsz

        # Encoder prefix
        input_tokens_enc = (
            self.input_tokens.unsqueeze(0).expand(old_bsz, -1).to(device)
        )
        temp_control_enc = self.multi_prefix[name]["wte_enc"](input_tokens_enc)
        temp_control_enc = self.prefix_dropout(temp_control_enc)
        past_key_values_enc = self.multi_prefix[name]["control_trans_enc"](
            temp_control_enc
        )  # bsz, seqlen, layer*emb
        past_key_values_enc = self.prefix_dropout(past_key_values_enc)
        bsz_enc, seqlen, _ = past_key_values_enc.shape
        past_key_values_enc = past_key_values_enc.view(
            bsz_enc,
            seqlen,
            self.match_n_layer * 2,
            self.match_n_head,
            self.match_n_embd,
        )
        past_key_values_enc = past_key_values_enc.permute([2, 0, 3, 1, 4]).split(2)

        result = []
        for i, key_val_enc in enumerate(past_key_values_enc):
            temp = dict()
            temp["encoder_prompt"] = {
                "prev_key": key_val_enc[0].contiguous(),
                "prev_value": key_val_enc[1].contiguous(),
                "prev_key_padding_mask": torch.zeros(bsz_enc, seqlen).to(device).bool(),
            }
            result.append(temp)

        return result

    def forward(
            self,
            input_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            token_type_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            head_mask: Optional[torch.Tensor] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            labels: Optional[torch.Tensor] = None,
            scores: Optional[torch.Tensor] = None,
            validation=False,
            **kwargs,
    ):
        assert self.toker != None

        bsz = input_ids.size(0)
        distribution = {}
        past_prompt = self.get_prompt("posterior", bsz, device=input_ids.device)
        outputs = self.bert(
            input_ids,
            past_prompt,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_hidden_states=True,
        )[0]
        masks = input_ids == 0 if token_type_ids == None else token_type_ids == 0
        pooler = outputs.masked_fill(masks.unsqueeze(-1), 0)
        pooler = pooler.sum(dim=1) / (1 - masks.long()).sum(dim=1).unsqueeze(-1)
        distribution["posterior"] = self.multi_prefix["posterior"]["mlp"](pooler).log_softmax(-1)

        loss = None
        if labels != None:
            distribution["prior"] = self.get_distribution(labels, bsz)
            if self.loss_fn == "rq":
                loss = self.comp_rq_loss(distribution)
            elif self.loss_fn == "qr":
                loss = self.comp_qr_loss(distribution)
            else: #Janson shanon divergence
                loss = self.comp_js_loss(distribution)
        elif scores != None:
            distribution["prior"] = scores
            if self.loss_fn == "rq":
                loss = self.comp_rq_loss(distribution)
            elif self.loss_fn == "qr":
                loss = self.comp_qr_loss(distribution)
            else: #Janson shanon divergence
                loss = self.comp_js_loss(distribution)
        res = {"dist": distribution["posterior"].exp(), "loss": loss}
        if not self.training and not validation:
            return res
        else:
            assert not validation
        return res

    ### compute loss
    def comp_rq_loss(self, distribution):
        posterior, prior = distribution["posterior"], distribution["prior"]
        l1, l2 = rq_loss(posterior, prior)
        l = l1.mean() - l2.mean()
        return l

    def comp_qr_loss(self, distribution):
        posterior, prior = distribution["posterior"], distribution["prior"]
        l1, l2 = qr_loss(posterior, prior)
        l = l1.mean() - l2.mean()
        return l

    def comp_js_loss(self, distribution): # Jensen-Shannon divergence
        posterior, prior = distribution["posterior"], distribution["prior"]
        ar = comp_r(posterior, prior)
        mixture = 0.5 * (posterior + ar)
        l_posterior = comp_kl_r(posterior, mixture)
        l_prior = comp_kl_r(ar, mixture)
        return 0.5 * (l_posterior + l_prior)

    @torch.no_grad()
    def predict(
            self,
            input_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            token_type_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            head_mask: Optional[torch.Tensor] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            **kwargs,
    ):
        assert not self.training
        assert self.toker != None

        bsz = input_ids.size(0)
        past_prompt = self.get_prompt("posterior", bsz, device=input_ids.device)
        outputs = self.bert(
            input_ids,
            past_prompt,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_hidden_states=True,
        )[0]

        masks = input_ids == 0 if token_type_ids == None else token_type_ids == 0
        pooler = outputs.masked_fill(masks.unsqueeze(-1), 0)
        pooler = pooler.sum(dim=1) / (1 - masks.long()).sum(dim=1).unsqueeze(-1)
        distribution = self.multi_prefix["posterior"]["mlp"](pooler).softmax(-1)

        return distribution

In [ ]:
#@title data utils

import gzip
import json
import os
import math
import random
import pickle
from functools import partial
from torch.utils.data import DataLoader, Sampler


def _norm(s):
    return ' '.join(s.strip().split())


class BucketSampler(Sampler):
    """
    this sampler will sort data by sequence length
    """

    def __init__(self, lens, bucket_size, batch_size, droplast=False, shuffle=True):
        self._lens = lens
        self._batch_size = batch_size
        self._bucket_size = bucket_size
        self._droplast = droplast
        self._shuf = shuffle

    def __iter__(self):
        ids = list(range(len(self._lens)))
        if self._shuf:
            random.shuffle(ids)
        buckets = [sorted(ids[i:i + self._bucket_size], key=lambda i: self._lens[i], reverse=True) for i in
                   range(0, len(ids), self._bucket_size)]
        batches = [bucket[i:i + self._batch_size] for bucket in buckets for i in
                   range(0, len(bucket), self._batch_size)]
        if self._droplast:
            batches = [batch for batch in batches if len(batch) == self._batch_size]
        if self._shuf:
            random.shuffle(batches)
        return iter(batches)

    def __len__(self):
        bucket_sizes = ([self._bucket_size]
                        * (len(self._lens) // self._bucket_size)
                        + [len(self._lens) % self._bucket_size])
        if self._droplast:
            return sum(s // self._batch_size for s in bucket_sizes)
        else:
            return sum(math.ceil(s / self._batch_size) for s in bucket_sizes)


class BucketingDataLoader(object):
    def __init__(self, toker, feature_dataset, batch_size, bucket=100, shuffle=True, **kwargs):
        assert "inputter_name" in kwargs, "inputter name should be provided"
        inputter_name = kwargs["inputter_name"]
        assert inputter_name in ["off", "mic", "esc", "single"], "undefined inputter name"
        if inputter_name == "off":
            assert "fine_task" in kwargs, "offensive fine task should be provided"
            fine_task = kwargs["fine_task"]
            with open(f'dataset/off/{fine_task}_train.pkl', 'rb') as f:
                self.data = pickle.load(f)
        elif inputter_name == "mic":
            with open(f'dataset/mic/mic_data_train.pkl', 'rb') as f:
                self.data = pickle.load(f)
        else:
            assert "fold_num" in kwargs, "fold number should be provided when inputter name is esc"
            fold_num = kwargs["fold_num"]
            valid_ids = json.load(open(f'{PARENT}dev_index.json', 'r'))[fold_num]
            with open(f'{PARENT}{inputter_name}_data.pkl', 'rb') as f:
                self.data = [line for line in pickle.load(f) if line.dialog_id not in valid_ids]

        self.toker = toker
        self.feature_dataset = feature_dataset
        self.batch_size = batch_size
        self.bucket_size = bucket * batch_size
        self.shuffle = shuffle

    def __iter__(self):
        trunc_chunk = []
        lens = []
        for feat in self.data:
            trunc_chunk.append(feat)
            lens.append(feat.input_length)

        dataset = self.feature_dataset(trunc_chunk)
        sampler = BucketSampler(lens, self.bucket_size, self.batch_size, droplast=True, shuffle=self.shuffle)
        loader = DataLoader(dataset, batch_sampler=sampler, num_workers=0,  # can be multi-worker
                            collate_fn=partial(self.feature_dataset.collate, toker=self.toker))
        yield from loader

    def __len__(self):
        return len(self.data)


class DistributedBucketingDataLoader(BucketingDataLoader):
    """ distributed version """

    def __init__(self, rank, num_replica, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rank = rank
        self.num_replica = num_replica
        self.data = self.data[self.rank::self.num_replica]


In [ ]:
#@title data esc (smallest)

import json
import tqdm
import random
import torch
import pickle
from typing import List
from math import ceil
from functools import partial
from numpy.random import triangular

from torch.utils.data import DataLoader, Sampler, Dataset
from torch.nn.utils.rnn import pad_sequence
from transformers.tokenization_utils import PreTrainedTokenizer

import logging

logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)


class Inputter(object):

    def __init__(self):
        # prepare
        self.convert_data_to_inputs = convert_data_to_inputs
        self.convert_inputs_to_features = convert_inputs_to_features

        # train
        self.train_dataset = FeatureDataset
        self.train_dataloader = BucketingDataLoader
        self.train_distributed_dataloader = DistributedBucketingDataLoader

        # valid
        self.valid_dataloader = DynamicBatchingLoader

        # infer
        self.prepare_infer_batch = prepare_infer_batch
        self.infer_dataloader = get_infer_batch


# basic utils
class InputFeatures(object):
    def __init__(self, dialog_id, input_ids, token_type_ids, label):
        self.dialog_id = dialog_id
        self.input_ids = input_ids
        self.input_length = len(input_ids)
        self.token_type_ids = token_type_ids
        self.label = label


def featurize(bos, eos, inputs, max_input_length=512):
    dialog_id, context, response = inputs["dialog_id"], inputs["context"], inputs["response"]
    label = inputs["label"]
    response = response + [eos]
    response_ids = response[:max_input_length]
    context_ids = [bos] + context + [bos]
    input_ids = context_ids + response_ids
    input_ids = input_ids[-max_input_length:]
    token_type_ids = [0] * len(input_ids)
    token_type_ids[-len(response_ids):] = [1] * len(response_ids)

    assert len(input_ids) == len(token_type_ids)

    return InputFeatures(dialog_id, input_ids, token_type_ids, label)


# for training
def convert_data_to_inputs(data, toker: PreTrainedTokenizer, **kwargs):
    assert 'max_input_length' in kwargs, "max_input_length should be given"
    process = lambda x: toker.convert_tokens_to_ids(
        toker.tokenize(x, max_length=kwargs['max_input_length'], truncation=True)
    )
    token_num = toker.vocab_size
    fine_to_coarse = {
        0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 13: 0, 14: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 2, 10: 2, 11: 2, 12: 2, -100: -100
    }

    context = _norm(' '.join(data["context"]))
    context = process(context)
    dialog_id = data["dialog_id"]
    responses = data['responses']
    inputs = []

    for reply_anno in responses:
        fine_ids = [process(f"[{anno}]")[0] - token_num if anno != "-" else -100 for anno in reply_anno[1:]]
        inputs.append({
            "dialog_id": dialog_id,
            "context": context,
            "response": process(_norm(reply_anno[0])),
            "label": [fine_to_coarse[id] for id in fine_ids],
        })

    return inputs


def convert_inputs_to_features(inputs, toker: PreTrainedTokenizer, **kwargs):
    if len(inputs) == 0:
        return []

    assert kwargs.get('max_input_length', None) != None, 'you should give max_input_length'
    max_input_length = kwargs.get('max_input_length')

    pad = toker.pad_token_id
    if pad is None:
        pad = toker.eos_token_id
        assert pad != None, 'either pad_token_id or eos_token_id should be provided'
    bos = toker.bos_token_id
    if bos is None:
        bos = toker.cls_token_id
        assert bos != None, 'either bos_token_id or cls_token_id should be provided'
    eos = toker.eos_token_id
    if eos is None:
        eos = toker.sep_token_id
        assert eos != None, 'either eos_token_id or sep_token_id should be provided'
    features = []
    for i, ipt in enumerate(inputs):
        feat = featurize(bos, eos, ipt, max_input_length)
        features.append(feat)

    return features


# for training
class FeatureDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, i):
        return self.features[i]

    def __len__(self):
        return len(self.features)

    @staticmethod
    def collate(features: List[InputFeatures], toker: PreTrainedTokenizer, infer=False):
        pad = toker.pad_token_id
        if pad is None:
            pad = toker.eos_token_id
            assert pad != None, 'either pad_token_id or eos_token_id should be provided'
        bos = toker.bos_token_id
        if bos is None:
            bos = toker.cls_token_id
            assert bos != None, 'either bos_token_id or cls_token_id should be provided'
        eos = toker.eos_token_id
        if eos is None:
            eos = toker.sep_token_id
            assert eos != None, 'either eos_token_id or sep_token_id should be provided'

        input_ids = pad_sequence(
            [torch.tensor(f.input_ids, dtype=torch.long) for f in features], batch_first=True, padding_value=pad)
        attention_mask = input_ids == pad
        token_type_ids = pad_sequence(
            [torch.tensor(f.token_type_ids, dtype=torch.long) for f in features], batch_first=True, padding_value=pad)

        labels = pad_sequence([torch.tensor(f.label, dtype=torch.long) for f in features], batch_first=True)

        res = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "labels": labels,
        }

        return res


# for dev
class DynamicBatchingLoader(object):
    """ this loader takes raw text file, used for validate perplexity """

    def __init__(self, toker, batch_size, **kwargs):
        assert "fold_num" in kwargs, "fold_num should be given"
        fold_num = kwargs["fold_num"]
        dev_index = json.load(open(f"{PARENT}dev_index.json", 'r'))[fold_num]
        with open(f"{PARENT}esc_data.pkl", 'rb') as f:
            self.data = pickle.load(f)
        self.trunc_chunk = []
        self.lens = []
        for feat in self.data:
            if feat.dialog_id in dev_index:
                self.trunc_chunk.append(feat)
                self.lens.append(feat.input_length)

        self.toker = toker
        self.bs = batch_size
        self.num_examples = len(self.trunc_chunk)
        self.kwargs = kwargs

    def __iter__(self, epoch=1):
        if epoch > 0:
            for i_epoch in range(epoch):
                yield from self._iter_epoch()
        else:
            while True:
                yield from self._iter_epoch()

    def __len__(self):
        return ceil(self.num_examples / self.bs)

    def _iter_epoch(self):
        try:
            features = []
            for feature in self.trunc_chunk:
                features.append(feature)
                if len(features) >= self.bs:
                    batch = self._batch_feature(features)
                    yield batch
                    features = []

        except StopIteration:
            pass

    def _batch_feature(self, features):
        return FeatureDataset.collate(features, self.toker)


# for inference
def convert_infer_to_features(inputs, toker: PreTrainedTokenizer, **kwargs):
    if len(inputs) == 0:
        return {}

    assert kwargs.get('max_input_length', None) != None, 'you should give max_input_length'
    max_input_length = kwargs.get('max_input_length')

    pad = toker.pad_token_id
    if pad is None:
        pad = toker.eos_token_id
        assert pad != None, 'either pad_token_id or eos_token_id should be provided'
    bos = toker.bos_token_id
    if bos is None:
        bos = toker.cls_token_id
        assert bos != None, 'either bos_token_id or cls_token_id should be provided'
    eos = toker.eos_token_id
    if eos is None:
        eos = toker.sep_token_id
        assert eos != None, 'either eos_token_id or sep_token_id should be provided'

    input_ids = [ipt + [eos] for ipt in inputs["inp_seq"]]
    input_ids = [ipt[-max_input_length:] for ipt in input_ids]

    decoder_input_ids = inputs["out_seq"]
    bos_tensor = torch.ones(decoder_input_ids.size(0), 1, dtype=decoder_input_ids.dtype) * bos
    decoder_input_ids = torch.cat((bos_tensor, decoder_input_ids), -1)

    features = {
        "input_ids": input_ids,
        "decoder_input_ids": decoder_input_ids,
    }
    return features


# for inference
def prepare_infer_batch(features, pad=0):
    input_ids = pad_sequence([torch.tensor(f.input_ids, dtype=torch.long) for f in features], batch_first=True,
                             padding_value=pad)
    attention_mask = input_ids == pad
    token_type_ids = pad_sequence([torch.tensor(f.token_type_ids, dtype=torch.long) for f in features],
                                  batch_first=True, padding_value=pad)

    res = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    }

    res['batch_size'] = res['input_ids'].size(0)

    return res


def get_infer_batch(corpus, toker, **kwargs):
    assert "max_input_length" in kwargs, "max_input_length should be given"
    process = lambda x: toker.convert_tokens_to_ids(
        toker.tokenize(x, max_length=kwargs["max_input_length"], truncation=True)
    )

    if "no_bar_info" in kwargs:
        bar = enumerate(corpus)
    else:
        bar = tqdm.tqdm(enumerate(corpus), total=len(corpus), desc="preference score computing")

    for sample_id, line in bar:
        data = json.loads(line)
        context = _norm(' '.join(data["context"]))
        context = process(context)
        responses = data['responses']
        inputs = []
        for reply in responses:
            inputs.append({
                "dialog_id": sample_id,
                "context": context,
                "response": process(_norm(reply)),
                "label": None,
            })
        features = convert_inputs_to_features(inputs, toker, **kwargs)

        yield prepare_infer_batch(features, toker.pad_token_id), sample_id

In [ ]:
#@title preference modelling
def compute_scores(preference_model_args, args, **dataloader_kwargs):
    mode = preference_model_args.mode
    kwargs = {'mode': mode}
    toker, model = build_pref_model(
        checkpoint=os.path.join(args.preference_model_dir, "best.bin"),
        local_rank=args.local_rank,
        args=preference_model_args,
        **kwargs,
    )
    deploy_model(model, args)
    model.eval()

    input_file = os.path.join(args.candidate_dir, "candidates.txt")
    preference_mark = args.preference_model_dir.split('/')[-1]
    save_path = os.path.join(args.candidate_dir, f"preference_score_{preference_mark}.npy")

    inputter = inputters["esc"]()
    with open(input_file, 'r', encoding='UTF-8') as f:
        corpus = f.readlines()
    infer_dataloader = inputter.infer_dataloader(
        toker=toker,
        corpus=corpus,
        batch_size=args.eval_batch_size,
        **dataloader_kwargs,
    )

    predictions = []
    for batch, sample_idx in infer_dataloader:
        batch = {k: v.to(args.device) if isinstance(v, Tensor) else v for k, v in batch.items()}
        scores = model.predict(**batch)[:, -1].cpu().numpy()
        predictions.append(scores)

    np.save(save_path, predictions)


In [ ]:
#@title model builder for data processor and preference modeling, Don't run otherwise!!
import json
import os
import logging
import torch

# This one should already be defined above as expected
# try:
#     from preference_modeling.models import models
# except ModuleNotFoundError:
#     from models import models


from transformers import (AutoTokenizer, AutoModel, AutoConfig)

logger = logging.getLogger(__name__)


def boolean_string(s):
    if s.lower() not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s.lower() == 'true'


def build_model(only_toker=False, checkpoint=None, args=None, **kwargs):
    config_file = f"{PARENT}preference_model.json"
    if not os.path.exists(config_file):
        raise ValueError

    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)

    if 'model_name' not in config or 'pretrained_model_path' not in config:
        raise ValueError
    toker = AutoTokenizer.from_pretrained(config['pretrained_model_path'])

    if only_toker:
        if 'expanded_vocab' in config:
            toker.add_tokens(config['expanded_vocab'], special_tokens=True)
        return toker

    # assert "mode" in kwargs   #already taken care of
    # mode = kwargs.pop("mode")
    # Model = models[mode]
    model = Model.from_pretrained(config['pretrained_model_path'], args)
    model.classifier = None
    for name, param in model.named_parameters():
        if name.startswith('bert.'):
            param.requires_grad = False

    if config.get('custom_config_path', None) != None:
        model = model(AutoConfig.from_pretrained(config['custom_config_path']))

    if 'gradient_checkpointing' in config:
        setattr(model.config, 'gradient_checkpointing', config['gradient_checkpointing'])

    if 'expanded_vocab' in config:
        toker.add_tokens(config['expanded_vocab'], special_tokens=True)
    model.tie_tokenizer(toker)

    if checkpoint != None:
        logger.info('loading finetuned model from %s' % checkpoint)
        model.load_state_dict(torch.load(checkpoint, map_location=torch.device('cpu')), strict=False)

    return toker, model

In [ ]:
#@title eval model
import torch
import logging
from torch import Tensor
import numpy as np
from collections import defaultdict
from sklearn import metrics

logger = logging.getLogger(__name__)


def cal_entropy(generated):
    etp_score = [0.0, 0.0, 0.0, 0.0]
    div_score = [0.0, 0.0, 0.0, 0.0]
    counter = [defaultdict(int), defaultdict(int), defaultdict(int), defaultdict(int)]
    for gg in generated:
        g = gg.rstrip().split()
        for n in range(4):
            for idx in range(len(g) - n):
                ngram = ' '.join(g[idx:idx + n + 1])
                counter[n][ngram] += 1
    for n in range(4):
        total = sum(counter[n].values()) + 1e-10
        for v in counter[n].values():
            etp_score[n] += - (v + 0.0) / total * (np.log(v + 0.0) - np.log(total))
        div_score[n] = (len(counter[n].values()) + 0.0) / total
    return etp_score, div_score


def eval_model(model, eval_dataloader, infer, args):
    logger.info('\ncompute eval model loss, using eval mode, '
                'please change it back to train after calling this function')
    model.eval()
    tot_loss = 0
    tot_sample = 0
    pointwise_loss = []
    with torch.no_grad():
        for step, batch in enumerate(eval_dataloader):
            batch = {k: v.to(args.device) if isinstance(v, Tensor) else v for k, v in batch.items()}

            outputs = model(**batch)
            loss_sample = outputs["loss"]
            if torch.isnan(loss_sample).sum().cpu().long().numpy() > 0:
                print(loss_sample)
                exit()
            num_sample = batch["input_ids"].shape[0]
            tot_loss += loss_sample.sum().item()
            tot_sample += num_sample

            if infer:
                pointwise_loss.extend(loss_sample.sum(dim=-1).cpu().tolist())
    mean_loss = tot_loss / tot_sample
    return mean_loss, pointwise_loss

In [ ]:
#@title train d-pm model

import argparse
import numpy as np
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from os.path import join
import datetime
from tqdm import tqdm
import time
from torch import Tensor

parser = argparse.ArgumentParser()
parser.add_argument("--task_name", type=str, default="esc", choices=["esc", "mic", "off"])
parser.add_argument("--mode", type=str, default="d-PM", choices=["d-PM", "soft", "major", "single"])
parser.add_argument("--fold_num", type=int, default=1)
parser.add_argument("--class_num", type=int, default=3)
parser.add_argument("--off_fine_task", type=str, default="offensive", choices=["offensive", "hate", "aggressive"])

parser.add_argument("--preseqlen", type=int, default=10)
parser.add_argument("--d_prefix", type=int, default=512)
parser.add_argument("--prefix_dropout", type=float, default=0.2)
parser.add_argument("--max_grad_norm", type=float, default=1.0)
parser.add_argument("--epoch_num", type=int, default=6)
parser.add_argument("--train_batch_size", type=int, default=32)
parser.add_argument("--eval_batch_size", type=int, default=64)
parser.add_argument("--gradient_accumulation_steps", type=int, default=4)
parser.add_argument("--valid_step", type=int, default=20)#800

parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--learning_rate", type=float, default=1e-4)
parser.add_argument("--warmup_steps", type=int, default=400)

parser.add_argument("--fp16", action="store_true")
parser.add_argument("--loss_scale", type=float, default=0.0)
parser.add_argument("--pbar", type=bool, default=True)
parser.add_argument("--chinese", action="store_true")
parser.add_argument("--local_rank", type=int, default=-1)
parser.add_argument("--load_checkpoint", type=str, default=None)
parser.add_argument("--loss_fn", type=str, default="js", choices=["qr", "rq", "js"])

print("js loss training")
args, _ = parser.parse_known_args()
parser = argparse.ArgumentParser()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.device = device
n_gpu = torch.cuda.device_count()
CACHE_EMPTY_STEP = 10000

names = {
        "mode": args.mode,
        "inputter_name": args.task_name,
        "fold_num": args.fold_num,
        "fine_task": args.off_fine_task,
    }

inputter = Inputter()

toker = build_model(only_toker=True)

train_dataloader = inputter.train_dataloader(
            toker=toker,
            feature_dataset=inputter.train_dataset,
            batch_size=args.train_batch_size,
            **names
        )

if args.epoch_num != None:
        args.num_optim_steps = args.epoch_num * (len(train_dataloader) // args.train_batch_size
                                                 + int(len(train_dataloader) % args.train_batch_size != 0))

eval_dataloader_loss = inputter.valid_dataloader(
        toker=toker,
        batch_size=args.eval_batch_size,
        **names
    )


#model and optimizer
_, model = build_model(checkpoint=args.load_checkpoint, args=args, **names)
model = model.to(device)
# params = [p.data for p in model.parameters()]  #only for distributed env
# all_reduce_and_rescale_tensors(params, float(torch.distributed.get_world_size()))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
total_params = sum([np.prod(p.size()) for p in model_parameters])

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'ln', 'LayerNorm.weight']  # no decay for bias and LayerNorm (ln)
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if p.requires_grad and not any(nd in n for nd in no_decay)],
      'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if p.requires_grad and any(nd in n for nd in no_decay)],
      'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, )
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=args.num_optim_steps
)

if args.fp16:
    logger.info('in fp16, using FusedAdam')
    from apex import amp

    model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

timestamp = datetime.datetime.now().strftime('%Y-%m%d-%H%M')[2:]
output_dir = OUTPUT   #can add timestep if wanted more organized folds nd files


train_logger = open(join(output_dir, 'train_log.csv'), 'a+', buffering=1)
eval_logger = open(join(output_dir, 'eval_log.csv'), 'a+', buffering=1)
print('epoch\tglobal_step\tstep\ttmp_loss\tmean_loss\tepoch_time', file=train_logger)
print('epoch\tglobal_step\tstep\tfreq_loss', file=eval_logger)


global_step = 0
lowest_eval_loss = 1
step = 0
epoch = 0

pbar = tqdm(total=args.num_optim_steps, desc=f"training", position=0)

while True:
      model.train()
      tr_loss, mean_loss, nb_tr_examples, nb_tr_steps = 0.0, 0.0, 0, 0
      train_start_time_epoch = time.time()
      for batch in train_dataloader:
          batch = {k: v.to(device) if isinstance(v, Tensor) else v for k, v in batch.items()}

          batch.update({'global_step': global_step})
          batch.update({'epoch': epoch})
          batch.update({'warmup_steps': args.warmup_steps})
          outputs = model(**batch)

          loss = outputs["loss"]
          predictions = outputs["dist"].cpu()
          if 'input_ids' in batch:
              input_ids = batch['input_ids']
          elif 'tgt_input_ids' in batch:
              input_ids = batch['tgt_input_ids']
          else:
              assert 'src_input_ids' in batch
              input_ids = batch['src_input_ids']

          if n_gpu > 1:
              loss = loss.mean()
          loss = loss / (args.train_batch_size * args.gradient_accumulation_steps / input_ids.shape[0])

          if args.fp16:
              with amp.scale_loss(loss, optimizer) as scaled_loss:
                  scaled_loss.backward()
          else:
              loss.backward()

          tmp_loss = float(loss.item()) * (
                  args.train_batch_size * args.gradient_accumulation_steps / input_ids.shape[0])
          tr_loss += tmp_loss
          nb_tr_examples += input_ids.size(0)
          nb_tr_steps += 1
          mean_loss = tr_loss / nb_tr_steps

          # gradient update
          step += 1
          if step % args.gradient_accumulation_steps == 0:
              if args.fp16:
                  torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
              else:
                  torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
              """
              if args.local_rank != -1:
                  grads = [p.grad.data for p in model.parameters()
                            if p.requires_grad and p.grad != None]
                  all_reduce_and_rescale_tensors(grads, float(1))
              """

              optimizer.step()
              scheduler.step()
              model.zero_grad()
              global_step += 1


              epoch_time = time.time() - train_start_time_epoch
              pbar_str = ''  # f"tok/s: {n_token_real_all_proc//epoch_time//1000}k "
              pbar_str += f"loss: {tmp_loss:.2f} "
              pbar_str += f"mean loss: {mean_loss:.2f} epoch: {epoch + 1}"

              pbar.set_postfix_str(pbar_str)
              if args.epoch_num != None:
                  pbar.update(args.gradient_accumulation_steps)
              else:
                  pbar.update(1)

              print(f'{epoch + 1}\t{global_step}\t{step}\t{tmp_loss}\t{mean_loss}\t{epoch_time}',
                    file=train_logger)

              if global_step % args.valid_step == 0:  # and epoch > 0:
                  # only rank 0 process evaluate
                  eval_loss, *_, = eval_model(
                      model=model,
                      eval_dataloader=eval_dataloader_loss,
                      infer=False,
                      args=args,
                  )
                  logger.info(f'**Eval (step {global_step})** eval_loss: {eval_loss}')
                  print(f'{epoch + 1}\t{global_step}\t{step + 1}\t{eval_loss}', file=eval_logger)
                  logger.info('current learning rate: ' + str(optimizer.param_groups[0]['lr']))

                  if eval_loss < lowest_eval_loss:
                      torch.save(model.state_dict(), join(output_dir, f'model_step-{global_step}.bin'))
                      torch.save(model.state_dict(), join(output_dir, 'best.bin'))
                      toker.save_vocabulary(output_dir)
                      try:
                          model.config.to_json_file(join(output_dir, f'config.json'))
                      except AttributeError:
                          model.module.config.to_json_file(join(output_dir, f'config.json'))
                      lowest_eval_loss = eval_loss
                  model.train()

              if args.epoch_num is None and global_step >= args.num_optim_steps:
                  break

          if (step + 1) % CACHE_EMPTY_STEP == 0:
              torch.cuda.empty_cache()
      epoch += 1

      if epoch == args.epoch_num:
        break
pbar.close()
train_logger.close()
eval_logger.close()

js loss training


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of Model were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['multi_prefix.posterior.control_trans_enc.0.bias', 'multi_prefix.posterior.control_trans_enc.0.weight', 'multi_prefix.posterior.control_trans_enc.2.bias', 'multi_prefix.posterior.control_trans_enc.2.weight', 'multi_prefix.posterior.mlp.0.bias', 'multi_prefix.posterior.mlp.0.weight', 'multi_prefix.posterior.mlp.2.bias', 'multi_prefix.posterior.mlp.2.weight', 'multi_prefix.posterior.wte_enc.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of Adam

################################################################################

#******************************************************************************

###############################################################################
# Alignment training

## alignment strategy:

first using pretrained model we create candidates

once candidates created, we score them using the preference model

once scored, the dataset is created using the candidates and scores.